In [44]:
#importacion de librerias
import pandas as pd
import os 
from pathlib import Path
import utils
from sklearn.model_selection import train_test_split
import spacy

In [45]:
#Uncomment this the first time u are running the repo
#!python -m spacy download en_core_web_sm

In [46]:
#carga de datos 
data_path = Path(os.getcwd()).parent / "data"
train_path = data_path / "bronze" / "dreaddit-train.csv"
test_path = data_path / "bronze" / "dreaddit-test.csv"

train = pd.read_csv(train_path, usecols=['text', 'id', 'label'])
test =  pd.read_csv(test_path, usecols=['text', 'id', 'label'])
df = pd.concat([train,test]).reset_index(drop=True)
df["text"] = df["text"].str.lower()

In [47]:
print(f"Hay {df.shape[0]} registros en el dataset")

Hay 3553 registros en el dataset


In [48]:
#Se filtran los textos con solo una palabra
df["token_count"] = df.text.apply(lambda x: len(x.split()))
ind_tk = df["token_count"] ==1
df = df[~ind_tk]

In [49]:
# Cargar el modelo de SpaCy
nlp = spacy.load('en_core_web_sm')
# se almacenan todos los textos en una lista
texts = df['text'].tolist()

# Procesar los textos en lotes
docs = nlp.pipe(texts)
#Se remueven las stopwords
df["clean_text"]  = [utils.remove_stopwords(doc,lemma=False,remove_stop=True) for doc in docs]

In [50]:

#Se eliminan los carateres especiales
df["clean_text"]= df["clean_text"].apply(utils.clean_text)
#Se eliminan los carateres especiales pero se mantienen los que son cruciales para indicar la separacion de oraciones.
df["clean_text_sentence_sep"]= df["clean_text"].apply(utils.clean_text,keep="sen", remove_words =True)


In [51]:
#Se separan los datos en test y train
df_train, df_test= train_test_split(df, test_size=0.1, random_state=42)

In [52]:
#Se exportan los datos procesados  a la siguiente capa 'silver'
data_path = Path(os.getcwd()).parent / "data"
train_output_path = data_path / "silver" / "dreadditCleanTrain.csv"
test_output_path = data_path / "silver" / "dreadditCleanTest.csv"

df_train.to_csv(train_output_path, index=False)
df_test.to_csv(test_output_path, index= False)